<a href="https://colab.research.google.com/github/rprimi/colB5BERT/blob/main/python/colB5BERT_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **colB5BERT:** Fine tuning colBERT with Big Five dataset

* Notebook inspirado no buscador denso de Leandro Carísio Fernandes

In [77]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip3 install transformers


In [ ]:
!git clone https://github.com/rprimi/colB5BERT.git

%cd /content/colB5BERT
!git pull


### Libraries

In [4]:
import sys
sys.path.append('/content/colB5BERT/python/')

import utils
import vsm
import sst

import os
import pandas as pd
import numpy as np
import textwrap
import pickle
import h5py
import logging

from transformers import BertModel, BertTokenizer
from transformers import RobertaModel, RobertaTokenizer

import torch
from torch.nn.functional import cosine_similarity
from tqdm import tqdm

import pandas as pd

from pathlib import Path
from sklearn.model_selection import train_test_split


from transformers import AutoTokenizer
from torch.utils import data
from torch.utils.data import DataLoader
from transformers import BatchEncoding

import torch
from transformers import AutoModel

from transformers import Trainer, TrainingArguments
from transformers import get_linear_schedule_with_warmup, get_cosine_with_hard_restarts_schedule_with_warmup, AdamW
from tqdm.auto import tqdm

In [80]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Jun 27 01:48:52 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    53W / 400W |  37955MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [84]:
df_test = pd.read_csv('/content/drive/MyDrive/colB5BERT/dataset_test_positive.tsv', sep='\t')
df_train = pd.read_csv('/content/drive/MyDrive/colB5BERT/dataset_train_positive.tsv', sep='\t')

len(df_test)
len(df_train)

# Sample data to test the pipeline

#df_test = df_test.sample(frac=0.08)
#df_train = df_train.sample(frac=0.08)
len(df_test)

df_test = df_test.sample(n = 1000)
df_train = df_train.sample(n = 1000)

In [85]:
queries_train = df_train['item_pt_text'].tolist()
docs_train = df_train['texto_dividido'].tolist()

queries_val = df_test['item_pt_text'].tolist()
docs_val = df_test['texto_dividido'].tolist()


## Fine-tuning dos encoders

Define os datasets e dataloaders:

In [86]:
max_length = 512
batch_size = 10
epochs = 4
lr = 2e-5


In [89]:
# Definição do Dataset
class Dataset(data.Dataset):
    # Recebe apenas um vetor de textos
    def __init__(self, tokenizer, textos, max_seq_length = max_length):
        self.max_seq_length = max_seq_length
        self.tokenizer = tokenizer
        self.textos = textos
        self.cache = {}

    def __len__(self):
        return len(self.textos)

    def __getitem__(self, idx):
        # Guarda os itens tokenizados num dict e apenas recupera de lá, pra não
        # ter que ficar tokenizando a cada época
        # Como estamos guardando no dict e idx é um slice, é necessário converter ele pra algo
        # mapeável
        self.cache[str(idx)] = self.cache.get(str(idx),
                   self.tokenizer(self.textos[idx],
                                  padding=True,
                                  truncation=True,
                                  max_length=self.max_seq_length
                                  )
                   )
        return self.cache[str(idx)]



In [91]:
# Create datasets
bert_weights_name = 'neuralmind/bert-base-portuguese-cased'
tokenizer = AutoTokenizer.from_pretrained(bert_weights_name)

dataset_queries_train = Dataset(tokenizer, queries_train)
dataset_docs_train = Dataset(tokenizer, docs_train)

dataset_queries_val = Dataset(tokenizer, queries_val)
dataset_docs_val = Dataset(tokenizer, docs_val)


def collate_fn(batch):
    #print('Dentro de collate_fn')
    #print(BatchEncoding(tokenizer.pad(batch, return_tensors='pt')))
    return BatchEncoding(tokenizer.pad(batch, return_tensors='pt'))

dataloader_queries_train = DataLoader(dataset_queries_train, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
dataloader_docs_train = DataLoader(dataset_docs_train, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

dataloader_queries_val = DataLoader(dataset_queries_val, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
dataloader_docs_val = DataLoader(dataset_docs_val, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)



Carrega os modelos:

In [92]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Se tiver que treinar os modelos, abre
model_query = AutoModel.from_pretrained(bert_weights_name).to(device)
model_doc = AutoModel.from_pretrained(bert_weights_name).to(device)

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertModel: ['cls.predicti

Define função pro cálculo da loss (modifiquei essa função para trabalhar com um vetor indicando quais pares são relevantes e quais não são:

In [105]:
epsilon = 1e-8

# Essa função já considera o resultado via batchs:
def compute_loss_com_gradiente(model_query, tokenized_queries, model_doc, tokenized_docs):
    outputs_queries = model_query(**tokenized_queries.to(device))
    outputs_docs    = model_doc(**tokenized_docs.to(device))


    # Extrai a última camada oculta associada ao token [CLS]
    tcls_queries = outputs_queries.last_hidden_state[:, 0, :]
    tcls_docs    = outputs_docs.last_hidden_state[:, 0, :]

    print(tcls_queries)
    print(tcls_docs)
    # Normaliza os tensores
    #tcls_queries = tcls_queries / torch.norm(tcls_queries, dim=1, keepdim=True)
    #tcls_docs = tcls_docs / torch.norm(tcls_docs, dim=1, keepdim=True)

    # Agora é necessário calcular a loss. Para isso, o primeiro passo é
    # calcular a similaridade entre uma query e documento (sim(q, d))
    similaridade = torch.matmul(tcls_queries, torch.transpose(tcls_docs, 0, 1))
    print(similaridade)
    # Calcula a exponencial da similaridade
    exp_sim = torch.exp(similaridade)

    # Calcula a loss
    soma_linhas = exp_sim.sum(dim=1) # Isso é pro denominador, inclui os exemplos positivos e negativos
    diagonal = torch.diag(exp_sim)
    log_loss = -1* torch.log(diagonal/(soma_linhas+epsilon))

    loss = torch.mean(log_loss)
    return loss

def compute_loss_sem_gradiente(model_query, tokenized_queries, model_doc, tokenized_docs):
    with torch.no_grad():
        return compute_loss_com_gradiente(model_query, tokenized_queries, model_doc, tokenized_docs)

def compute_loss_dataloaders(model_query, dataloader_query, model_doc, dataloader_docs):
    loss = 0
    n_batches = 0
    for batch_query, batch_docs in zip(dataloader_query, dataloader_docs):
        loss = loss + compute_loss_sem_gradiente(model_query, batch_query, model_doc, batch_docs)
        n_batches += 1
    return loss/n_batches

In [106]:
%%time
# Só pra medir o tempo que ele demora para calcular a loss em todo o dataset de treinamento
model_query.eval()
model_doc.eval()
print(f'Loss de treinamento: {compute_loss_dataloaders(model_query, dataloader_queries_train, model_doc, dataloader_docs_train)}')
print(f'Loss de validação: {compute_loss_dataloaders(model_query, dataloader_queries_val, model_doc, dataloader_docs_val)}')

tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]], device='cuda:0')
tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]], device='cuda:0')
tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, na

KeyboardInterrupt: ignored

Agora treina os dois encoders simulatenamente:

In [101]:
%%time

treinar_e_salvar_modelos = True

dir_modelos = '/content/drive/MyDrive/colB5BERT/check_point/'

if treinar_e_salvar_modelos:
  # Training loop
  optimizer_query = AdamW(model_query.parameters(), lr=lr)
  optimizer_doc = AdamW(model_doc.parameters(), lr=lr)

  num_training_steps = epochs * len(dataloader_queries_train)
  num_warmup_steps = int(num_training_steps * 0.1)

  # get_linear_schedule_with_warmup get_cosine_with_hard_restarts_schedule_with_warmup
  scheduler_query = get_cosine_with_hard_restarts_schedule_with_warmup(optimizer_query, num_warmup_steps, num_training_steps)
  scheduler_doc = get_cosine_with_hard_restarts_schedule_with_warmup(optimizer_doc, num_warmup_steps, num_training_steps)

  for epoch in tqdm(range(epochs), desc='Epochs'):
      model_query.train()
      model_doc.train()

      train_losses = []
      for batch_query, batch_docs in tqdm(list(zip(dataloader_queries_train, dataloader_docs_train)), mininterval=0.5, desc='Train', disable=False):
          optimizer_query.zero_grad()
          optimizer_doc.zero_grad()

          loss = compute_loss_com_gradiente(model_query, batch_query, model_doc, batch_docs)
          loss.backward()

          optimizer_query.step()
          optimizer_doc.step()

          scheduler_query.step()
          scheduler_doc.step()

      model_query.save_pretrained(f'{dir_modelos}{epoch}/query/')
      model_doc.save_pretrained(f'{dir_modelos}{epoch}/doc/')

      model_query.eval()
      model_doc.eval()

      print(f'Loss de treinamento {epoch}: {compute_loss_dataloaders(model_query, dataloader_queries_train, model_doc, dataloader_docs_train)}')
      print(f'Loss de validação {epoch}: {compute_loss_dataloaders(model_query, dataloader_queries_val, model_doc, dataloader_docs_val)}')

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epochs:   0%|          | 0/4 [00:00<?, ?it/s]

Train:   0%|          | 0/100 [00:00<?, ?it/s]

Loss de treinamento 0: nan
Loss de validação 0: nan


Train:   0%|          | 0/100 [00:00<?, ?it/s]

Loss de treinamento 1: nan
Loss de validação 1: nan


Train:   0%|          | 0/100 [00:00<?, ?it/s]

Loss de treinamento 2: nan
Loss de validação 2: nan


Train:   0%|          | 0/100 [00:00<?, ?it/s]

Loss de treinamento 3: nan
Loss de validação 3: nan
CPU times: user 2min 59s, sys: 2.38 s, total: 3min 2s
Wall time: 3min 5s
